In [199]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as DataLoader
import torch
from sklearn.datasets import make_classification
# import torchvision.datasets as datasets
# import torchvison.transforms as transforms

In [200]:
class CustomDataset:
    
    def __init__(self, data ,targets):
        
        self.data =data
        self.targets = targets
        
    def __len__ (self):
        
        return len(self.data)
    
    def __getitem__(self, idx):
        
        current_sample = self.data[idx,:]
        
        current_target = self.targets[idx]
        
        return {
            
            'sample' : torch.tensor(current_sample ,dtype = torch.float),
            'target' : torch.tensor(current_target ,dtype = torch.long)
        }

In [201]:
# classification data from sklearn
data , targets =  make_classification(n_samples=1000)

In [202]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_targets,test_targets = train_test_split(data,targets,stratify = targets)

In [203]:

# object of custom dataset class for training
train_dataset = CustomDataset(train_data,train_targets)


# object of custom dataset class for testing
test_dataset = CustomDataset(test_data,test_targets)

In [204]:
# loaders for training


train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=4,num_workers=2)

# loaders for testing
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=10,num_workers=2)

In [205]:
train_data.shape  # shape of training data

(750, 20)

In [206]:


# to access any elemnet from taring and testing dataset
train_dataset[90]

{'sample': tensor([-0.6627,  0.1625, -0.4478,  0.2608, -0.3235,  0.0207,  0.2220, -0.1722,
          1.4966,  0.1699, -0.4218,  3.9994,  2.4079,  0.4183,  1.3749,  2.7511,
         -1.6494, -1.1717, -2.1900, -1.4021]),
 'target': tensor(1)}

In [207]:

# oUr linear regression model

# y = x.w +b
model = lambda x,w,b: torch.matmul(x,w)+b  

### Run the below cell and read it's comments

In [208]:

# outputs = []

# labels = []

# with torch.no_grad():
#     for i,data in enumerate(train_loader):
        
#         # as we are having 750 examples in training data
#         # and our batch size is 100 for train_loader so our train_loader will create 7 mini batches for training
#         # and each batch will be a dictionary of sample and target ( sample and target have come from __getitem__ function from Custom Dataset class)
#         print(i,data,'\n')

In [209]:

# initialising weights raindomly

w = torch.rand(20,1,requires_grad=True)

b = torch.randn(1,requires_grad=True)

lr = 0.01

for epoch in range(10):
    
    epoch_loss = 0
    
    for data in train_loader:
        
        # as we are having 750 examples in training data
        # and our batch size is 100 for train_loader so our train_loader will create 7 mini batches for training
        # and each batch will be a dictionary of sample and target ( sample and target have come from __getitem__ function from Custom Dataset class)
        x_train = data['sample']
        y_train = data['target']
        
        output = model(x_train,w,b)
        
        
        # uncommnent
#         print(f"Initially value of Gradient for Weights matrix ---------> {w.grad}")
        # uncomment the line and see the output  
        # we are getting predictions in batches of 100 as our batch size was 100
        
        # print(output.shape)
        
        loss = torch.mean((y_train.view(-1)-output.view(-1))**2)
         
        # for calculating loss per epoch
        epoch_loss += loss.item()
        
        
        loss.backward()
        
        #uncomment
#         print(f'After going backward ------------->  {w.grad}')
        
        
        with torch.no_grad():
            w = w - lr*w.grad
            b = b - lr*b.grad
            
        # we are again pls calculate grads for w again from now for 2,3,4,5 ... epochs  
        
        # requires_grad_  means it is inplace
        w.requires_grad_(True)
        b.requires_grad_(True)
        
    print('\n',epoch,epoch_loss,'\n')


 0 222.3605015259236 


 1 17.145906555466354 


 2 17.12337014079094 


 3 17.12742791697383 


 4 17.127609942108393 


 5 17.12761803623289 


 6 17.127618491649628 


 7 17.1276184739545 


 8 17.127618483267725 


 9 17.12761841248721 



In [210]:
outputs = []

true_labels = []


with torch.no_grad():
    for i in test_loader:

        x_test = i['sample']
        y_test = i['target']

        output = model(x_test,w,b)

        # true labels
        true_labels.append(y_test)

        outputs.append(output)


In [211]:
print(type(outputs)) # list of all tesnor of shape 10,1 ( 10 beacuse batch size was 10 from teest laoder)

<class 'list'>


In [212]:
# concatenate and make our output shape of 250,1
outputs = torch.cat(outputs).view(-1)

true_labels = torch.cat(true_labels).view(-1)

from sklearn.metrics import roc_auc_score

print(roc_auc_score(true_labels,outputs))

0.9575652841781873
